In [245]:
import dlib
import cv2
import os
import pandas as pd
import numpy as np

In [246]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [247]:
def convertToGray(img): # function that convert image to grayscale.
	return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [248]:
def get_landmarks(imag):
	landmarks = []
	detections = detector(imag, 1)
    
	for k,d in enumerate(detections): #For all detected face instances individually
		shape = predictor(imag, d) #Draw Facial Landmarks with the predictor class
		xlist = []
		ylist = []
		for i in range(0,68): #Store X and Y coordinates in two lists
			xlist.append(float(shape.part(i).x))
			ylist.append(float(shape.part(i).y))

		for x, y in zip(xlist, ylist): #Store all landmarks in one list in the format x1,y1,x2,y2,etc.
			landmarks.append(x)
			landmarks.append(y)
            
	if len(detections) > 0:
		return landmarks
	else: #If no faces are detected, return error message to other function to handle
		landmarks = "error"
		return landmarks

In [249]:
def extractFrames(pathIn, pathOut):
	df = pd.DataFrame()
	os.mkdir(pathOut)

	lista = []
	lines = []
	col = []
	i = 1
	for i in range(1, 10):
		lines.append("frame_%d" % i)
        #lines[i] = "land_%d" % i
		i += 1
	print(lines)
        
	j = 1
	for j in range(1, 137):
		col.append("land_%d" % j)
        #col[j] = "frame_%d" % j
		j += 1
	print(col)
    
	cap = cv2.VideoCapture(pathIn)
	count = 0
	bias = 50
	while (cap.isOpened() and count<10): # o contador<10 serve para tirar apenas os 10 primeiros frames do video

		# Capture frame-by-frame
		ret, frame = cap.read()

		if ret == True:
			print('Read %d frame: ' % count, ret)

			frame = convertToGray(frame) # convert image to gray
			# uses the classifier HaarCascade to extract the faces.
			haar_face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
			faces = haar_face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5) # detecta quantas faces ha na imagem

			for (x,y,w,h) in faces: # Para cada face faz um corte
				crop_img = frame[y-bias: y+h+bias, x-bias: x+w+bias]

			landmark = get_landmarks(crop_img) # Aqui recebo lista com landmarks.
			#print(type(landmark)) # Podemos ver com isso que a variavel retornada pela funcao get_landmarks eh do tipo list.
			#print(len(landmark)) # Tamanho da lista
			#array = np.array([landmark])
			
			land_array = pd.DataFrame([landmark], index = lines, columns = col)
			df = df.append(land_array)

			clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
			clahe_image = clahe.apply(crop_img) # Aplicação do Clahe a imagem ja em cinza
			detections = detector(clahe_image, 1)#Detect the faces in the image
			for k,d in enumerate(detections): #For each detected face
            			shape = predictor(clahe_image, d) #Get coordinates
            			for i in range(1,68): #There are 68 landmark points on each face
                     			#For each point, draw a red circle with thickness2 on the original frame
                    			cv2.circle(crop_img, (shape.part(i).x, shape.part(i).y), 1, (0,0,255), thickness=2)

			cv2.imwrite(os.path.join(pathOut, "frame{:d}.jpg".format(count)), crop_img)  # save crop_img as JPEG file

			count += 1
		else:
			break

	print(land_array)
	print(df)
	#print(df.shape)
	# When everything done, release the capture
	cap.release()
	#cv2.destroyAllWindows()


In [250]:
def createDataFrame():

	df = pd.DataFrame()
	print(df)

In [251]:
def main():
	extractFrames('Em1_Fala1_CarolinaHolly.mp4', 'A4')

In [252]:
if __name__=="__main__":
	main()

['frame_1', 'frame_2', 'frame_3', 'frame_4', 'frame_5', 'frame_6', 'frame_7', 'frame_8', 'frame_9']
['land_1', 'land_2', 'land_3', 'land_4', 'land_5', 'land_6', 'land_7', 'land_8', 'land_9', 'land_10', 'land_11', 'land_12', 'land_13', 'land_14', 'land_15', 'land_16', 'land_17', 'land_18', 'land_19', 'land_20', 'land_21', 'land_22', 'land_23', 'land_24', 'land_25', 'land_26', 'land_27', 'land_28', 'land_29', 'land_30', 'land_31', 'land_32', 'land_33', 'land_34', 'land_35', 'land_36', 'land_37', 'land_38', 'land_39', 'land_40', 'land_41', 'land_42', 'land_43', 'land_44', 'land_45', 'land_46', 'land_47', 'land_48', 'land_49', 'land_50', 'land_51', 'land_52', 'land_53', 'land_54', 'land_55', 'land_56', 'land_57', 'land_58', 'land_59', 'land_60', 'land_61', 'land_62', 'land_63', 'land_64', 'land_65', 'land_66', 'land_67', 'land_68', 'land_69', 'land_70', 'land_71', 'land_72', 'land_73', 'land_74', 'land_75', 'land_76', 'land_77', 'land_78', 'land_79', 'land_80', 'land_81', 'land_82', 'land_

ValueError: Shape of passed values is (1, 1), indices imply (9, 1)